In [ ]:

###########################################
#                  TP-IA                  #
##########################################


import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
###################################################
#              Importation de data                #
###################################################

#train and test data directory
data_dir = "/home/ajanif/Bureau/Martel/tiny-imagenet-200/train"
test_data_dir = "/home/ajanif/Bureau/Martel/tiny-imagenet-200/test"
val_data_dir="/home/ajanif/Bureau/Martel/tiny-imagenet-200/val"

#load the train and test data
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))


test_dataset = ImageFolder(test_data_dir,transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor() ]))



#--------------------------  declarer transform-----------

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4


img,label=dataset[1]

dataset.classes
##################################################
#       Test de l'mportation de données          #
##################################################

def display_img(img,label):
    plt.imshow(img.permute(1,2,0))
display_img(*dataset[100])




In [ ]:
#---------------------------------------------------------------------------------fin etape 1 -------------------------------------------------------------------


trainset = torchvision.datasets.ImageFolder(root='/home/ajanif/Bureau/Martel/tiny-imagenet-200/train',
                                         transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/home/ajanif/Bureau/Martel/tiny-imagenet-200/test', 
                                           transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
dataset_dir = os.path.join("./tiny-imagenet-200/")

val_dir = os.path.join(dataset_dir, 'val')

img_dir = os.path.join(val_dir, 'images')

fp = open(os.path.join(val_dir, 'val_annotations.txt'), 'r')

data = fp.readlines()

val_img_dict = {}

for line in data:

    words = line.split('\t')

val_img_dict[words[0]] = words[1]

fp.close()

#-----------------------------------
classes = dict()
fp = open(os.path.join(dataset_dir, 'words.txt'), 'r')

data = fp.readlines()

for line in data:

    words = line.strip('\n').split('\t')

classes[words[0]] = words[1].split(',')[0]

fp.close()


id_to_class = {i: c for c, i in testset.class_to_idx.items()}



def class_(id) :

    return classes[id_to_class[int(id)]]



def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()




# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{dataset.classes[j]:5s}' for j in range(batch_size)))



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

###############################################
#      Construction de réseau                 #
###############################################



class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 6)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(10, 20, 6)
        self.pool = nn.MaxPool2d(2, 2)
        
      
        
        ##self.fc1 = nn.Linear(2880,120 ) 
        ##self.fc2 = nn.Linear(120, 84)
        self.fc1 = nn.Linear(2880,120 ) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 200)
        self.fc4 = nn.Linear(200, 30)
        
        ##self.fc3 = nn.Linear(84, 200)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

################################################
#            Définir la fonction perte         #
################################################
import torch.optim as optim


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:

######################################################
#            Entraînement de réseau                  #
######################################################



for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')    #Fin d'entaînement


In [ ]:

#############################################
#           Sauvgardement de données        #
############################################
PATH = './yas.pth'   #chemin 

torch.save(net.state_dict(), PATH)


In [ ]:
#########################################################
#                    Accuracy                           #
#########################################################


correct =0
total =0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')